In [38]:
import pandas as pd

# Dependencies der Scripts
import pdfplumber
import pyjstat

# Die beste Gemeinde des Kantons Graubünden

## Quellen



In [47]:
!cd scrape && python extract-codes.py
!cd scrape && python fetch-districts.py


   Code      Gemeinde
0  3542  Albula/Alvra
1  3955     Landquart
2  3543        Surses
3  3701        Andeer
4  3513  Lantsch/Lenz
         Gemeinde  Strassenflaeche-Hektar
0     Vaz/Obervaz                      88
1    Lantsch/Lenz                      24
2  Schmitten (GR)                       7
3    Albula/Alvra                     100
4          Surses                     138


In [39]:
df_wiki = pd.read_csv('./data/scraped/wikipedia-length.csv')


In [40]:
df_accidents = pd.read_csv('./data/scraped/accidents.csv')


In [41]:
df_regionalportraits = pd.read_csv('./data/scraped/regionalportraits.csv')


In [42]:
df_streets = pd.read_csv('./data/scraped/streets.csv')


In [43]:
gemeinden_accidents = set(df_streets['Gemeinde'].unique())
gemeinden_wikipedia = set(df_wiki['Gemeinde'].unique())

# Unterschiede feststellen
nur_in_accidents = gemeinden_accidents - gemeinden_wikipedia
nur_in_wikipedia = gemeinden_wikipedia - gemeinden_accidents

print("Nur in Verkehrsunfall-Daten vorhanden:", nur_in_accidents)
print("Nur in Wikipedia-Daten vorhanden:", nur_in_wikipedia)

Nur in Verkehrsunfall-Daten vorhanden: set()
Nur in Wikipedia-Daten vorhanden: set()
